In [ ]:
!pip install -q sentence-transformers rouge-score nltk pandas openpyxl sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import math
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mapped_signals_deepSeek_A100.xlsx to mapped_signals_deepSeek_A100.xlsx
Saving mapped_signals_gemma_A100.xlsx to mapped_signals_gemma_A100.xlsx
Saving mapped_signals_llama_A100.xlsx to mapped_signals_llama_A100.xlsx
Saving PV_Signal_List_10_TEST_SCENARIOS_MAPPED_GOLD.xlsx to PV_Signal_List_10_TEST_SCENARIOS_MAPPED_GOLD.xlsx


In [ ]:
import pandas as pd
df_Gold_Standard = pd.read_excel("PV_Signal_List_10_TEST_SCENARIOS_MAPPED_GOLD.xlsx")
df_deepseek = pd.read_excel("mapped_signals_deepSeek_A100.xlsx")
df_Llama = pd.read_excel("mapped_signals_llama_A100.xlsx")
df_gemma = pd.read_excel("mapped_signals_gemma_A100.xlsx")
print("Preview of your data:")
print(df_Gold_Standard.head())
print(df_deepseek.head())
print(df_Llama.head())
print(df_gemma.head())

Preview of your data:
       Signal term Date detected                                     Status  \
0           Stroke        May-25                                    Ongoing   
1              SJS        Jun-25                                    Refuted   
2      Myocarditis        Apr-25   Closed as Potential Risk (Not Important)   
3  QT prolongation        Sep-24       Closed as Potential Risk (Important)   
4     Pancreatitis        Mar-25  Closed as Identified Risk (Not Important)   

  Date closed     Source of signal  \
0         NaN        Meta-analysis   
1      25-Aug         Case reports   
2      10-Jul             Registry   
3      12-Oct       Clinical Study   
4      09-Sep  Spontaneous Reports   

                                          Summary              Action  \
0                   Signal still under assessment             Pending   
1          Evaluation shows no causal association                 NaN   
2  Evidence suggests possible risk but low impact      

In [ ]:
df_combined = pd.DataFrame()
df_combined["Signal term"] = df_Gold_Standard["Signal term"]
df_combined["Gold"] = df_Gold_Standard["Mapped Sections"]

df_combined['DeepSeek'] = df_deepseek['Mapping']
df_combined['DeepSeek Gen time'] = df_deepseek['Gen Time']
df_combined['DeepSeek Token per Sec'] = df_deepseek['Tokens Per Sec']

df_combined['Llama'] = df_Llama['Mapping']
df_combined['Llama Gen time'] = df_Llama['Gen Time']
df_combined['Llama Token per Sec'] = df_Llama['Tokens Per Sec']

df_combined['gemma'] = df_gemma['Mapping']
df_combined['gemma Gen time'] = df_gemma['Gen Time']
df_combined['gemma Token per Sec'] = df_gemma['Tokens Per Sec']


In [ ]:
# Text-processing metric helpers
from nltk.tokenize import word_tokenize, sent_tokenize
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def word_count(text):
    if not isinstance(text, str): return 0
    return len(word_tokenize(text))

def sentence_count(text):
    if not isinstance(text, str): return 0
    return len(sent_tokenize(text))

def vocab_diversity(text):
    if not isinstance(text, str) or len(text.strip())==0: return 0.0
    toks = [t.lower() for t in word_tokenize(text) if any(c.isalnum() for c in t)]
    if len(toks)==0: return 0.0
    return len(set(toks)) / len(toks)

In [ ]:
# Semantic model for relevance and coherence
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Overall score weights (sum = 1.0)
WEIGHTS = {
    "relevance": 0.4,
    "completeness": 0.4,
    "coherence": 0.2
}

def semantic_similarity(a, b):
    """Return cosine similarity between sentences a and b (0..1)"""
    if not isinstance(a, str) or not isinstance(b, str):
        return 0.0
    if a.strip()=="" or b.strip()=="":
        return 0.0
    emb = embedder.encode([a, b], convert_to_tensor=True)
    return float(util.cos_sim(emb[0], emb[1]).item())

def coherence_score(text):
    """
    A simple coherence proxy: average cosine similarity between consecutive sentence embeddings.
    If only 1 sentence -> returns 1.0 (max).
    """
    if not isinstance(text, str) or text.strip()=="":
        return 0.0
    sents = sent_tokenize(text)
    if len(sents) <= 1:
        return 1.0  # single sentence assumed coherent by itself
    embs = embedder.encode(sents, convert_to_tensor=True)
    sims = []
    for i in range(len(sents)-1):
        sims.append(float(util.cos_sim(embs[i], embs[i+1]).item()))
    return float(np.mean(sims)) if sims else 0.0

# 7) Completeness via ROUGE-L F1 vs gold
def completeness_score(pred, gold):
    if not isinstance(pred, str) or not isinstance(gold, str):
        return 0.0
    try:
        score = rouge.score(pred, gold)
        # rougeL fmeasure
        return float(score['rougeL'].fmeasure)
    except Exception as e:
        return 0.0

# 8) Putting it together for a single text pair
def compute_metrics_for_pair(pred_text, gold_text, gen_time_s, tokens_per_s):
    wc = word_count(pred_text)
    sc = sentence_count(pred_text)
    vd = vocab_diversity(pred_text)
    rel = semantic_similarity(pred_text, gold_text)
    comp = completeness_score(pred_text, gold_text)
    coh = coherence_score(pred_text)
    # overall weighted
    overall = (rel * WEIGHTS['relevance'] +
               comp * WEIGHTS['completeness'] +
               coh * WEIGHTS['coherence'])
    return {
        "words": wc,
        "sentences": sc,
        "vocab_diversity": vd,
        "relevance": rel,
        "completeness": comp,
        "coherence": coh,
        "overall_score": overall,
        "gen_time_s": gen_time_s if gen_time_s is not None else np.nan,
        "tokens_per_s": tokens_per_s if tokens_per_s is not None else np.nan
    }


In [ ]:
results = []
for _, row in df_combined.iterrows():
    metrics = compute_metrics_for_pair(row['DeepSeek'], row['Gold'], row['DeepSeek Gen time'],row['DeepSeek Token per Sec'])
    results.append({
            "model": 'DeepSeek',
            **metrics
        })
    metrics = compute_metrics_for_pair(row['Llama'], row['Gold'], row['Llama Gen time'],row['Llama Token per Sec'])
    results.append({
            "model": 'Llama',
            **metrics
        })
    metrics = compute_metrics_for_pair(row['gemma'], row['Gold'], row['gemma Gen time'],row['gemma Token per Sec'])
    results.append({
            "model": 'gemma',
            **metrics
        })



results_df = pd.DataFrame(results)

# 10) Aggregate per model (mean)
agg = results_df.groupby('model').agg({
    'words': 'mean',
    'sentences': 'mean',
    'vocab_diversity': 'mean',
    'relevance': 'mean',
    'completeness': 'mean',
    'coherence': 'mean',
    'overall_score': 'mean',
    'gen_time_s': 'mean',
    'tokens_per_s': 'mean'
}).reset_index()

# Round for readability
agg = agg.round({
    'words':0, 'sentences':1,
    'vocab_diversity':3, 'relevance':3, 'completeness':3, 'coherence':3, 'overall_score':3,
    'gen_time_s':2, 'tokens_per_s':2
})

print("\nPer-model aggregated metrics:")
display(agg)



Per-model aggregated metrics:


,model,words,sentences,vocab_diversity,relevance,completeness,coherence,overall_score,gen_time_s,tokens_per_s
0,DeepSeek,273.0,14.0,0.579,0.554,0.127,0.285,0.329,16.14,18.59
1,Llama,134.0,4.0,0.503,0.766,0.349,0.558,0.557,5.76,26.54
2,gemma,165.0,6.9,0.455,0.663,0.321,0.369,0.467,143.95,6.49
